# Gemini API: Prompting with Video


### This notebook has been replaced by the [Video understanding](../quickstarts/Video_understanding.ipynb) one in which you'll learn about all the latest video understanding capabilities.

An archived version of this notebook can be found [here](https://github.com/google-gemini/cookbook/blob/gemini-1.5-archive/quickstarts/Video.ipynb).